# Week 2 Homework: Regression

## Data Preparation

Import necessary libraries such as `pandas` and `numpy`

In [1]:
import pandas as pd
import numpy as np

Download dataset from the link provided and place it under `data` folder

In [8]:
!wget -P data/ https://raw.githubusercontent.com/alexeygrigorev/datasets/master/laptops.csv

--2024-10-07 21:39:48--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/laptops.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... --2024-10-07 21:39:48--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/laptops.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 

connected.
HTTP request sent, awaiting response... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 298573 (292K) [text/plain]
Saving to: ‘data/laptops.csv’

laptops.csv           0%[                    ]       0  --.-KB/s               200 OK
Length: 298573 (292K) [text/plain]
Saving to: ‘data/laptops.csv’

laptops.csv           0%[                    ]       0  --.-KB/s               

laptops.csv         100%[===================>] 291.58K  1.28MB/s    in 0.2s    

2024-10-07 21:39:48 (1.28 MB/s) - ‘data/laptops.csv’ saved [298573/298573]

laptops.csv         100%[===================>] 291.58K  1.28MB/s    in 0.2s    

2024-10-07 21:39:48 (1.28 MB/s) - ‘data/laptops.csv’ saved [298573/298573]



Read the raw dataset

In [9]:
laptops_raw = pd.read_csv('data/laptops.csv')
laptops_raw.head()

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


Tidy the dataset by lowering the column names and replace any space with underscore to make it more readable

In [11]:
# tidy the dataset
laptops_raw.columns = laptops_raw.columns.str.lower().str.replace(' ', '_')
laptops_raw.head()

,laptop,status,brand,model,cpu,ram,storage,storage_type,gpu,screen,touch,final_price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


For this project, select the necessary columns below:
- `ram`
- `storage`
- `screen`
- `final_price`

In [12]:
laptops_df = laptops_raw[['ram','storage','screen','final_price']]
laptops_df.head()

,ram,storage,screen,final_price
0,8,512,15.6,1009.00
1,8,256,15.6,299.00
2,8,256,15.6,789.00
3,16,1000,15.6,1199.00
4,16,512,15.6,669.01


from the example of first five rows above, we can see that the `final_price` variable only have two digits after decimal point

## Questions

### Question 1
Check column with missing values

In [13]:
laptops_df.isnull().sum()

ram            0
storage        0
screen         4
final_price    0
dtype: int64

It is the `screen` column (option *C*)

### Question 2
Median for `RAM` column

In [17]:
np.median(laptops_df['ram'])

np.float64(16.0)

### Question 3

- deal with missing values for the `laptops` prepared dataset by fill it with 0 or with the mean of the corresponding variable.
- find which option that gives better RMSE


#### Setting up the validations framework

- Shuffle the dataset (the filtered one you created above), use seed `42`
- Split your data with distribution of
  - `train`: 60%
  - `val` : 20%
  - `test`20%

split the datasets

In [18]:
n = len(laptops_df)
n_val = n_test = int(n * 0.2)
n_train = n - n_val - n_test

n, n_train, n_val, n_test

(2160, 1296, 432, 432)

shuffle the datasets

In [47]:
idx = np.arange(n)

np.random.seed(42)
np.random.shuffle(idx)

df_train = laptops_df.iloc[idx[:n_train]]
df_val = laptops_df.iloc[idx[n_train:n_train+n_val]]
df_test = laptops_df.iloc[idx[n_train+n_val:]]

In [48]:
df_train.head()

,ram,storage,screen,final_price
2079,32,1000,15.6,1123.29
668,4,64,14.1,201.05
2073,32,1000,14.0,997.74
1113,16,512,13.3,1016.00
788,32,1000,16.0,2739.00


In [49]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

preparing the labels

In [50]:
y_train = (df_train.final_price.values)
y_val = (df_val.final_price.values)
y_test = (df_test.final_price.values)

deleting columns from the training and evaluations

In [51]:
del df_train['final_price']
del df_val['final_price']
del df_test['final_price']

#### Create necessary functions

function for linear regression

In [26]:
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]


function for RMSE

In [27]:
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

#### Dealing with missing values

Filling missing values with `mean` of `screen` variable and find the `RMSE`

In [52]:
mean_screen = df_train.screen.mean()

X_mean_train = df_train.copy()
X_mean_train['screen'] = X_mean_train.screen.fillna(mean_screen)
w_0_mean, w_mean = train_linear_regression(X_mean_train, y_train)

X_mean_val = df_val.copy()
X_mean_val['screen'] = X_mean_val.screen.fillna(mean_screen)

y_mean_pred_val = w_0_mean + X_mean_val.dot(w_mean)

rmse_mean = np.round(rmse(y_val, y_mean_pred_val),2)
print(f"RMSE value for filling with mean is: {rmse_mean}")

RMSE value for filling with mean is: 600.27
RMSE value for filling with mean is: 600.27


Filling missing values with `0` and find the `RMSE`

In [53]:
X_zero_train = df_train.copy()
X_zero_train['screen'] = X_zero_train.screen.fillna(0)
w_0_zero, w_zero = train_linear_regression(X_zero_train, y_train)

X_zero_val = df_val.copy()
X_zero_val['screen'] = X_zero_val.screen.fillna(0)

y_zero_pred_val = w_0_zero + X_zero_val.dot(w_zero)

rmse_zero = np.round(rmse(y_val, y_zero_pred_val),2)
print(f"RMSE value for filling with zero is: {rmse_zero}")

RMSE value for filling with zero is: 597.36
RMSE value for filling with zero is: 597.36


#### Conclusion

The `RMSE` value for filling missing data with `0` is **slightly better** than with `mean value`

### Question 4

- train a regularized linear regression
- For this question, fill the `NA` with `0`
- Try different values of `r` from this list: `[0, 0.01, 0.1, 1, 5, 10, 100]`

#### Defining the function

In [54]:
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

#### Iterate for each value of `r`

In [56]:
for r in [0, 0.01, 0.1, 1, 5, 10, 100]:
    w_0, w = train_linear_regression_reg(X_zero_train, y_train, r=r)
    y_zero_reg_val = w_0 + X_zero_val.dot(w)
    rmse_val = np.round(rmse(y_val, y_zero_reg_val),2)
    print(f'RMSE value of {r}: {rmse_val}')

RMSE value of 0: 597.36
RMSE value of 0.01: 597.36
RMSE value of 0.1: 597.35
RMSE value of 1: 597.21
RMSE value of 5: 597.01
RMSE value of 10: 597.06
RMSE value of 100: 597.9
RMSE value of 0: 597.36
RMSE value of 0.01: 597.36
RMSE value of 0.1: 597.35
RMSE value of 1: 597.21
RMSE value of 5: 597.01
RMSE value of 10: 597.06
RMSE value of 100: 597.9


`r` value of `0` gives the best RMSE score of `0.43`

### Question 5

- find out `standard deviation` for different degree of randomization
- Try different seed values of `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]`

In [57]:
rmse_list = []

for seeds in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]:

    n = len(laptops_df)
    n_val = n_test = int(n * 0.2)
    n_train = n - n_val - n_test
    
    idx = np.arange(n)
    np.random.seed(seeds)
    np.random.shuffle(idx)

    df_train = laptops_df.iloc[idx[:n_train]]
    df_val = laptops_df.iloc[idx[n_train:n_train+n_val]]
    df_test = laptops_df.iloc[idx[n_train+n_val:]]

    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)
    
    y_train = (df_train.final_price.values)
    y_val = (df_val.final_price.values)
    y_test = (df_test.final_price.values)
    
    del df_train['final_price']
    del df_val['final_price']
    del df_test['final_price']
    
    X_zero_train = df_train.copy()
    X_zero_train['screen'] = X_zero_train.screen.fillna(0)
    w_0, w = train_linear_regression(X_zero_train, y_train)
    
    X_zero_val = df_val.copy()
    X_zero_val['screen'] = X_zero_val.screen.fillna(0)

    y_zero_pred_val = w_0 + X_zero_val.dot(w)
    
    rmse_val = np.round(rmse(y_val, y_zero_pred_val),2)
    
    rmse_list.append(rmse_val)

In [58]:
rmse_list

[np.float64(565.45),
 np.float64(636.8),
 np.float64(588.96),
 np.float64(597.81),
 np.float64(571.96),
 np.float64(573.24),
 np.float64(647.34),
 np.float64(550.44),
 np.float64(587.33),
 np.float64(576.1)]

In [59]:
rmse_std = np.round(np.std(rmse_list),3)
print(f'the standard deviation of the RMSE values is: {rmse_std}')

the standard deviation of the RMSE values is: 29.176
the standard deviation of the RMSE values is: 29.176


### Question 6

- Split the dataset like previously, use seed 9.
- Combine train and validation datasets.
- Fill the missing values with 0 and train a model with r=0.001
- count the RMSE on the test dataset

In [64]:
n = len(laptops_df)
n_val = n_test = int(n * 0.2)
n_train = n - n_val - n_test

seeds = 9
r = 0.001

idx = np.arange(n)
np.random.seed(seeds)
np.random.shuffle(idx)


df_shuffled = laptops_df.iloc[idx]
    
df_train = df_shuffled.iloc[:n_train].copy()
df_val = df_shuffled.iloc[n_train:n_train+n_val].copy()
df_test = df_shuffled.iloc[n_train+n_val:].copy()

frames = [df_train, df_val]
df_train_val = pd.concat(frames)

df_train_val = df_train_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train_val = df_train_val.final_price.values
y_test = df_test.final_price.values

del df_train_val['final_price']
del df_test['final_price']

X_zero_train_val = df_train_val.copy()
X_zero_train_val['screen'] = X_zero_train_val.screen.fillna(0)
w_0, w = train_linear_regression_reg(X_zero_train_val, y_train_val, r=r)

X_zero_test = df_test.copy()
X_zero_test['screen'] = X_zero_test.screen.fillna(0)

y_zero_pred_test = w_0 + X_zero_test.dot(w)

rmse_test = np.round(rmse(y_test, y_zero_pred_test),2)

print(f'The RMSE value for test dataset is: {rmse_test}')

The RMSE value for test dataset is: 608.61
The RMSE value for test dataset is: 608.61
